<h1> SET-UP </h1>

<h3> DEFINITIONS </h3>

In [1]:
import json
import requests

RULES_URL = 'https://api.rules.art/graphql'

<h3> FUNCTIONS </h3>

In [2]:
def group_by_season_and_scarcity(card_list):
    # Grouping by season and scarcity
    cards = {}

    for card in card_list:
        if 'cardModel' in card:
            season = card['cardModel']['season']
            scarcity = card['cardModel']['scarcity']['name']
        else:
            season = card['season']
            scarcity = card['scarcity']['name']

        key = (season, scarcity)  # Use a tuple as a key to group by both season and scarcity

        if key not in cards:
            cards[key] = []
        cards[key].append(card)

    return cards


In [3]:
def all_cards_model():
    query = f"""
    {{
        allCardModels{{
            slug,
            uid,
            scarcity{{
                name
            }},
            season,
            artistName
        }}
    }}
    """

    r = requests.post(RULES_URL, json={'query': query})
    #print(r.status_code)

    try:
        r_data = r.json()
        rules_data = r_data.get('data', {}).get('allCardModels')
        return group_by_season_and_scarcity(rules_data)
        

    except ValueError:
        raise ValueError("Error: Invalid JSON response from the API.")

    

In [4]:
def owned_cards(account_name):
  
  query = f"""
  {{
    user(slug: "{account_name}") {{
      ownedCardModels {{
        cardModel {{
          slug,
          uid,
          scarcity {{
            name
          }}
          artistName,
          season
        }},
        serialNumbers
      }}
    }}
  }}
  """
  r = requests.post(RULES_URL, json={'query': query})
  #print(r.status_code)

  try:
    r_data = r.json()
    user = r_data.get('data', {}).get('user')
    #Check if user is found
    if user is None:
       print("User not found")
       return []
    else: 
      user_data = user.get('ownedCardModels')
      #print(user_data)
      return group_by_season_and_scarcity(user_data)
  except ValueError:
      raise ValueError("Error: Invalid JSON response from the API.")
  
  


In [5]:
def find_doublons(card_list):
    doublons = {}

    # Iterate over each season's card list
    for season, cards in card_list.items():
        filtered_cards = [card for card in cards if len(card['serialNumbers']) > 1]
        if filtered_cards:
            doublons[season] = filtered_cards

    return doublons


In [6]:
def find_missing_cards(card_list_1, card_list_2):
    # Extract uids from the first output
    list_1 = set(item.get('uid', item.get('cardModel', {}).get('uid')) for sublist in card_list_1.values() for item in sublist)

    # Extract uids from the second output
    list_2 = set(item['cardModel']['uid'] for sublist in card_list_2.values() for item in sublist)

    # Find the uids that are present in the first output but not in the second output
    uids_not_in_second_list = list_1 - list_2

    # Convert list_1 back to a dictionary using a dictionary comprehension
    list_1_dict = {key: value for key, value in card_list_1.items()}

    # Filter the elements from the first output based on the uids not present in the second output
    elements_not_in_second_output = {key: [item for item in sublist if item.get('uid', item.get('cardModel', {}).get('uid')) in uids_not_in_second_list] for key, sublist in list_1_dict.items()}

    return elements_not_in_second_output


In [7]:
def get_lowest_serial_numbers(data):
    # Create a dictionary to store the lowest serial numbers for each slug
    lowest_serial_numbers = {}

    # Iterate through the data dictionary
    for key, items_list in data.items():
        lowest_serial_numbers[key] = []
        for item in items_list:
            # Extract the slug and serialNumbers from the item
            slug = item['cardModel']['slug']
            uid = item['cardModel']['uid']
            scarcity = item['cardModel']['scarcity']
            artist_name = item['cardModel']['artistName']
            serial_numbers = item['serialNumbers']

            # Check if serialNumbers is empty
            if serial_numbers:
                # Find the lowest serial number for the current slug
                lowest_serial = min(serial_numbers)
                # Append the data for the current slug to the output
                output_item = {
                    'cardModel': {
                        'slug': slug,
                        'uid': uid,
                        'scarcity': scarcity,
                        'artistName': artist_name
                    },
                    'serialNumbers': [lowest_serial]
                }
                lowest_serial_numbers[key].append(output_item)

    # Remove keys with no valid data
    lowest_serial_numbers = {k: v for k, v in lowest_serial_numbers.items() if v}

    return lowest_serial_numbers

In [8]:
def find_common_card_models(dict1, dict2):
    common_card_models = {}

    for key in dict1.keys():
        if key in dict2:
            list1 = dict1[key]
            list2 = dict2[key]

            for item1 in list1:
                for item2 in list2:
                    if item1['cardModel'] == item2['cardModel']:
                        card_model_key = f"{item1['cardModel']['slug']}_{item1['cardModel']['scarcity']['name']}"
                        if card_model_key not in common_card_models:
                            common_card_models[card_model_key] = {
                                'cardModel': {
                                    'slug': item1['cardModel']['slug'],
                                    'uid': item1['cardModel']['uid'],
                                    'scarcity': item1['cardModel']['scarcity'],
                                    'artistName': item1['cardModel']['artistName'],
                                    'season': item1['cardModel']['season']
                                },
                                'serialNumbers': []
                            }
                        serial_numbers = []
                        if item1['serialNumbers']:
                            serial_numbers.extend(item1['serialNumbers'])
                        if item2['serialNumbers']:
                            serial_numbers.extend(item2['serialNumbers'])
                        lowest_serial_number = min(serial_numbers)
                        if lowest_serial_number in item2['serialNumbers']:
                            common_card_models[card_model_key]['serialNumbers'] = [lowest_serial_number]
                        else:
                            del common_card_models[card_model_key]
                        break  # Exit inner loop if a matching cardModel is found in list2

    return group_by_season_and_scarcity(list(common_card_models.values()))

In [9]:
def print_good_format(card_list):
    # Printing the data in the desired format

    if all(not slugs for slugs in card_list.values()):
        print("No card to display")
        return
    
    for season_scarcity, slugs in card_list.items():
        print(f"SEASON {season_scarcity[0]} - {season_scarcity[1].capitalize()}:")
        for slug in slugs:
            if 'cardModel' in slug:
                serial_numbers = slug['serialNumbers']
                artist_name = slug['cardModel']['artistName']
                scarcity_name = slug['cardModel']['scarcity']['name']
                uid = slug['cardModel']['uid']
                print(f"{uid} - {artist_name} - #{serial_numbers}")
            elif 'slug' in slug:
                artist_name = slug['artistName']
                scarcity_name = slug['scarcity']['name']
                uid = slug['uid']
                print(f"{uid} - {artist_name}")
            
        print()  # Empty line between seasons



***

***

<h1> RULES - ALL CARDS </h1>

In [10]:
all_cards = all_cards_model()
#print(f"{all_cards}\n")
print_good_format(all_cards)

SEASON 1 - Common:
1 - Le Règlement
3 - Caballero
5 - JeanJass
7 - Zuukou Mayzie
9 - Benjamin Epps
11 - thaHomey
13 - So La Lune
15 - Sheldon
17 - Youv Dee
19 - Livaï
21 - Flem
23 - Spider Zed
25 - Lycos
27 - Winnterzuko
29 - Jewel Usain
31 - Slimka
33 - 8ruki
35 - Mahdi Ba
37 - Jwles
39 - Azur
41 - Moji x Sboy
43 - Rowjay
45 - Zinée
47 - Skuna
51 - Di-Meh
53 - JMK$
49 - Deen Burbigo
56 - Kerchak
58 - Doums
60 - Bigflo
62 - Ysos

SEASON 1 - Platinium:
2 - Le Règlement
4 - Caballero
6 - JeanJass
8 - Zuukou Mayzie
10 - Benjamin Epps
12 - thaHomey
14 - So La Lune
16 - Sheldon
18 - Youv Dee
20 - Livaï
22 - Flem
24 - Spider Zed
26 - Lycos
28 - Winnterzuko
30 - Jewel Usain
32 - Slimka
34 - 8ruki
36 - Mahdi Ba
38 - Jwles
40 - Azur
42 - Moji x Sboy
44 - Rowjay
46 - Zinée
48 - Skuna
52 - Di-Meh
54 - JMK$
50 - Deen Burbigo
57 - Kerchak
59 - Doums
61 - Bigflo
63 - Ysos
55 - Caba. & J.J.

SEASON 1 - Halloween:
75 - Zinée
67 - Zuukou Mayzie
72 - Youv Dee
69 - JeanJass
68 - Caballero
71 - Le Règleme

***

***

<h1> MAIN ACCOUNT </h1>

In [11]:
main_account = input("What is your main account profile name ? ").casefold().replace(" ", "")
print(f"Account: {main_account.upper()}")
card_list_main_account = owned_cards(main_account)
#print(json.dumps(card_list_main_account, indent=2))
print(f"{card_list_main_account}\n")

Account: FOGUETE
{(1, 'halloween'): [{'cardModel': {'slug': 'zinee-season-1-halloween', 'uid': 75, 'scarcity': {'name': 'halloween'}, 'artistName': 'Zinée', 'season': 1}, 'serialNumbers': [1842]}, {'cardModel': {'slug': 'jeanjass-season-1-halloween', 'uid': 69, 'scarcity': {'name': 'halloween'}, 'artistName': 'JeanJass', 'season': 1}, 'serialNumbers': [2081]}, {'cardModel': {'slug': 'caballero-season-1-halloween', 'uid': 68, 'scarcity': {'name': 'halloween'}, 'artistName': 'Caballero', 'season': 1}, 'serialNumbers': [1763]}], (1, 'common'): [{'cardModel': {'slug': 'flem-season-1-common', 'uid': 21, 'scarcity': {'name': 'common'}, 'artistName': 'Flem', 'season': 1}, 'serialNumbers': [2808, 3107, 3464, 3873, 3883]}, {'cardModel': {'slug': 'le-reglement-season-1-common', 'uid': 1, 'scarcity': {'name': 'common'}, 'artistName': 'Le Règlement', 'season': 1}, 'serialNumbers': [1928, 3827]}, {'cardModel': {'slug': 'lycos-season-1-common', 'uid': 25, 'scarcity': {'name': 'common'}, 'artistName'

<h3> ALL CARDS OWNED </h3>

In [12]:
print_good_format(card_list_main_account)

SEASON 1 - Halloween:
75 - Zinée - #[1842]
69 - JeanJass - #[2081]
68 - Caballero - #[1763]

SEASON 1 - Common:
21 - Flem - #[2808, 3107, 3464, 3873, 3883]
1 - Le Règlement - #[1928, 3827]
25 - Lycos - #[3051, 3385]
58 - Doums - #[1938, 2627]
43 - Rowjay - #[1998, 3770]
23 - Spider Zed - #[3287]
53 - JMK$ - #[3391]
31 - Slimka - #[3274]
29 - Jewel Usain - #[3227]
9 - Benjamin Epps - #[3623]
15 - Sheldon - #[2923, 3129]
39 - Azur - #[3614]
62 - Ysos - #[2309, 2437]
47 - Skuna - #[3682]
27 - Winnterzuko - #[3622]
60 - Bigflo - #[2579]
33 - 8ruki - #[3966]
51 - Di-Meh - #[3873]
41 - Moji x Sboy - #[3718]
49 - Deen Burbigo - #[3926]
56 - Kerchak - #[3101]
17 - Youv Dee - #[4067]
45 - Zinée - #[4034]
35 - Mahdi Ba - #[4119]
11 - thaHomey - #[4152]
5 - JeanJass - #[4604]
19 - Livaï - #[4455]

SEASON 1 - Platinium:
30 - Jewel Usain - #[261]
10 - Benjamin Epps - #[181]

SEASON 2 - Common:
106 - thaHomey - #[281]
80 - JeanJass - #[282]
110 - 404Billy - #[415]
96 - Spider Zed - #[202]
82 - So La

<h3> DOUBLONS </h3>

In [13]:
doublons_1 = find_doublons(card_list_main_account)

#print(doublons)

print_good_format(doublons_1)

SEASON 1 - Common:
21 - Flem - #[2808, 3107, 3464, 3873, 3883]
1 - Le Règlement - #[1928, 3827]
25 - Lycos - #[3051, 3385]
58 - Doums - #[1938, 2627]
43 - Rowjay - #[1998, 3770]
15 - Sheldon - #[2923, 3129]
62 - Ysos - #[2309, 2437]



<h3> MISSING CARDS </h3>

In [14]:
#print(f"{all_cards}\n")
#print(f"{card_list_main_account}\n")
print(find_missing_cards(all_cards, card_list_main_account))
print_good_format(find_missing_cards(all_cards, card_list_main_account))


{(1, 'common'): [{'slug': 'caballero-season-1-common', 'uid': 3, 'scarcity': {'name': 'common'}, 'season': 1, 'artistName': 'Caballero'}, {'slug': 'zuukou-mayzie-season-1-common', 'uid': 7, 'scarcity': {'name': 'common'}, 'season': 1, 'artistName': 'Zuukou Mayzie'}, {'slug': 'so-la-lune-season-1-common', 'uid': 13, 'scarcity': {'name': 'common'}, 'season': 1, 'artistName': 'So La Lune'}, {'slug': 'jwles-season-1-common', 'uid': 37, 'scarcity': {'name': 'common'}, 'season': 1, 'artistName': 'Jwles'}], (1, 'platinium'): [{'slug': 'le-reglement-season-1-platinium', 'uid': 2, 'scarcity': {'name': 'platinium'}, 'season': 1, 'artistName': 'Le Règlement'}, {'slug': 'caballero-season-1-platinium', 'uid': 4, 'scarcity': {'name': 'platinium'}, 'season': 1, 'artistName': 'Caballero'}, {'slug': 'jeanjass-season-1-platinium', 'uid': 6, 'scarcity': {'name': 'platinium'}, 'season': 1, 'artistName': 'JeanJass'}, {'slug': 'zuukou-mayzie-season-1-platinium', 'uid': 8, 'scarcity': {'name': 'platinium'}, 

***

***

<h1> SECOND ACCOUNT </h1> 

In [15]:
second_account = input("What is your second account profile name ? ").casefold().replace(" ", "")
print(f"Account: {second_account.upper()}")
card_list_second_account = owned_cards(second_account)
#print(json.dumps(card_list_main_account, indent=2))
print(card_list_second_account)

Account: VINCE0602
{(1, 'common'): [{'cardModel': {'slug': 'lycos-season-1-common', 'uid': 25, 'scarcity': {'name': 'common'}, 'artistName': 'Lycos', 'season': 1}, 'serialNumbers': [3946]}, {'cardModel': {'slug': 'di-meh-season-1-common', 'uid': 51, 'scarcity': {'name': 'common'}, 'artistName': 'Di-Meh', 'season': 1}, 'serialNumbers': [3934]}, {'cardModel': {'slug': 'jmks-season-1-common', 'uid': 53, 'scarcity': {'name': 'common'}, 'artistName': 'JMK$', 'season': 1}, 'serialNumbers': [3844, 4601]}, {'cardModel': {'slug': '8ruki-season-1-common', 'uid': 33, 'scarcity': {'name': 'common'}, 'artistName': '8ruki', 'season': 1}, 'serialNumbers': [4080]}, {'cardModel': {'slug': 'flem-season-1-common', 'uid': 21, 'scarcity': {'name': 'common'}, 'artistName': 'Flem', 'season': 1}, 'serialNumbers': [4192, 4202]}, {'cardModel': {'slug': 'winnterzuko-season-1-common', 'uid': 27, 'scarcity': {'name': 'common'}, 'artistName': 'Winnterzuko', 'season': 1}, 'serialNumbers': [4076, 4151, 4298]}, {'card

<h3> ALL CARDS OWNED </h3>

In [16]:
print_good_format(card_list_second_account)

SEASON 1 - Common:
25 - Lycos - #[3946]
51 - Di-Meh - #[3934]
53 - JMK$ - #[3844, 4601]
33 - 8ruki - #[4080]
21 - Flem - #[4192, 4202]
27 - Winnterzuko - #[4076, 4151, 4298]
15 - Sheldon - #[4072]
17 - Youv Dee - #[4130, 4185]
49 - Deen Burbigo - #[4138, 4208]
23 - Spider Zed - #[4184, 4216]
45 - Zinée - #[4241, 4566]
58 - Doums - #[3520]
1 - Le Règlement - #[4411, 4636]
43 - Rowjay - #[4623]
60 - Bigflo - #[3647]
56 - Kerchak - #[3666]
11 - thaHomey - #[4492]

SEASON 2 - Common:
110 - 404Billy - #[1060, 1109, 1269]
80 - JeanJass - #[]
82 - So La Lune - #[884, 1280]
118 - Baby Neelou - #[1061]
98 - Wallace Cleaver - #[]
108 - Selug & $enar - #[924]
96 - Spider Zed - #[]
106 - thaHomey - #[]
100 - BabySolo33 - #[886, 921, 1143, 1280]
116 - Roshi - #[]
86 - Winnterzuko - #[1050, 1182]
84 - Deen Burbigo - #[]
112 - HOUDI - #[]
94 - Youv Dee - #[]
102 - Karmen - #[]
114 - Limsa d'Aulnay - #[]
124 - Rad Cartier - #[]

SEASON 2 - Holo:
113 - HOUDI - #[]
83 - So La Lune - #[]



<h3> DOUBLONS </h3>

In [17]:
doublons_2 = find_doublons(card_list_second_account)

print(doublons_2)

print_good_format(doublons_2)

{(1, 'common'): [{'cardModel': {'slug': 'jmks-season-1-common', 'uid': 53, 'scarcity': {'name': 'common'}, 'artistName': 'JMK$', 'season': 1}, 'serialNumbers': [3844, 4601]}, {'cardModel': {'slug': 'flem-season-1-common', 'uid': 21, 'scarcity': {'name': 'common'}, 'artistName': 'Flem', 'season': 1}, 'serialNumbers': [4192, 4202]}, {'cardModel': {'slug': 'winnterzuko-season-1-common', 'uid': 27, 'scarcity': {'name': 'common'}, 'artistName': 'Winnterzuko', 'season': 1}, 'serialNumbers': [4076, 4151, 4298]}, {'cardModel': {'slug': 'youv-dee-season-1-common', 'uid': 17, 'scarcity': {'name': 'common'}, 'artistName': 'Youv Dee', 'season': 1}, 'serialNumbers': [4130, 4185]}, {'cardModel': {'slug': 'deen-burbigo-season-1-common', 'uid': 49, 'scarcity': {'name': 'common'}, 'artistName': 'Deen Burbigo', 'season': 1}, 'serialNumbers': [4138, 4208]}, {'cardModel': {'slug': 'spider-zed-season-1-common', 'uid': 23, 'scarcity': {'name': 'common'}, 'artistName': 'Spider Zed', 'season': 1}, 'serialNumb

<h3> MISSING CARDS </h3>

In [18]:
print_good_format(find_missing_cards(all_cards, card_list_second_account))

SEASON 1 - Common:
3 - Caballero
5 - JeanJass
7 - Zuukou Mayzie
9 - Benjamin Epps
13 - So La Lune
19 - Livaï
29 - Jewel Usain
31 - Slimka
35 - Mahdi Ba
37 - Jwles
39 - Azur
41 - Moji x Sboy
47 - Skuna
62 - Ysos

SEASON 1 - Platinium:
2 - Le Règlement
4 - Caballero
6 - JeanJass
8 - Zuukou Mayzie
10 - Benjamin Epps
12 - thaHomey
14 - So La Lune
16 - Sheldon
18 - Youv Dee
20 - Livaï
22 - Flem
24 - Spider Zed
26 - Lycos
28 - Winnterzuko
30 - Jewel Usain
32 - Slimka
34 - 8ruki
36 - Mahdi Ba
38 - Jwles
40 - Azur
42 - Moji x Sboy
44 - Rowjay
46 - Zinée
48 - Skuna
52 - Di-Meh
54 - JMK$
50 - Deen Burbigo
57 - Kerchak
59 - Doums
61 - Bigflo
63 - Ysos
55 - Caba. & J.J.

SEASON 1 - Halloween:
75 - Zinée
67 - Zuukou Mayzie
72 - Youv Dee
69 - JeanJass
68 - Caballero
71 - Le Règlement
66 - Eden Dillinger
70 - Winnterzuko
73 - Vladimir Cauchemar 

SEASON 2 - Common:
78 - Caballero
90 - Kerchak
76 - Le Règlement
104 - Mehdi Maïzi
88 - Vladimir Cauchemar
120 - Mairo
130 - Stony Stone
92 - Zuukou Mayzie


***

***

<h1> SUMMARY </h1>

<h3> MISSING CARDS </h3>
New cards existing in the second account but not in the main one
<p><i> If multiple card model are new, then the lowest serial is selected </i></p>

In [19]:
not_in_main = find_missing_cards(card_list_second_account, card_list_main_account)
not_in_main = get_lowest_serial_numbers(not_in_main)
print(f"{not_in_main}\n")
print_good_format(not_in_main)

{}

No card to display


<h3> LOWER SERIAL </h3>
Cards that has a lower serial in the second account than in the main one

In [20]:
lower_serial = find_common_card_models(card_list_main_account, card_list_second_account)
print(f"{lower_serial}\n")
print_good_format(lower_serial)

{}

No card to display


<h3><b> TO SEND (from second to main account) </b></h3>
Cards to send from second to main because new or lower serial than an existing one

In [21]:
# Combine the datasets
to_send = {}

# Merge first dataset into combined_data
for key, value in not_in_main.items():
    to_send.setdefault(key, []).extend(value)

# Merge second dataset into combined_data
for key, value in lower_serial.items():
    to_send.setdefault(key, []).extend(value)

# Print the combined data
#print(to_send)

print_good_format(to_send)

No card to display


***
***

<h3><b> NOT USEFULL CARDS </b></h3>

In [22]:
def dict_substraction(card_list_1, card_list_2):
    # Step 1: Create dictionaries with uid as the key and serialNumbers as the value for both datasets
    serial_numbers_data1 = {}
    for key, card_list in card_list_1.items():
        serial_numbers_data1.update({card['cardModel']['uid']: card['serialNumbers'] for card in card_list})

    serial_numbers_data2 = {}
    for key, card_list in card_list_2.items():
        serial_numbers_data2.update({card['cardModel']['uid']: card['serialNumbers'] for card in card_list})

    # Step 2: Perform the subtraction for each uid
    result_data = {}
    for uid, serial_numbers in serial_numbers_data1.items():
        serial_numbers_to_subtract = serial_numbers_data2.get(uid, [])
        result_serial_numbers = list(set(serial_numbers) - set(serial_numbers_to_subtract))

        if result_serial_numbers:
            result_data[uid] = result_serial_numbers

    # Step 3: Create the final result in the original format
    result = {}
    for key, card_list in card_list_1.items():
        result[key] = [{'cardModel': card['cardModel'], 'serialNumbers': result_data[card['cardModel']['uid']]} for card in card_list if card['cardModel']['uid'] in result_data]
    
    return result

lowest_doublons_main = get_lowest_serial_numbers(doublons_1)

first_account = dict_substraction(doublons_1, lowest_doublons_main)
second_account = dict_substraction(card_list_second_account, to_send)

# Combine the datasets
not_usefull = {}

# Merge first dataset into combined_data
for key, value in first_account.items():
    not_usefull.setdefault(key, []).extend(value)

# Merge second dataset into combined_data
for key, value in second_account.items():
    not_usefull.setdefault(key, []).extend(value)

# Print the combined data
#print(not_usefull)

print_good_format(not_usefull)


SEASON 1 - Common:
21 - Flem - #[3464, 3873, 3107, 3883]
1 - Le Règlement - #[3827]
25 - Lycos - #[3385]
58 - Doums - #[2627]
43 - Rowjay - #[3770]
15 - Sheldon - #[3129]
62 - Ysos - #[2437]
25 - Lycos - #[3946]
51 - Di-Meh - #[3934]
53 - JMK$ - #[4601, 3844]
33 - 8ruki - #[4080]
21 - Flem - #[4192, 4202]
27 - Winnterzuko - #[4298, 4076, 4151]
15 - Sheldon - #[4072]
17 - Youv Dee - #[4185, 4130]
49 - Deen Burbigo - #[4208, 4138]
23 - Spider Zed - #[4184, 4216]
45 - Zinée - #[4241, 4566]
58 - Doums - #[3520]
1 - Le Règlement - #[4411, 4636]
43 - Rowjay - #[4623]
60 - Bigflo - #[3647]
56 - Kerchak - #[3666]
11 - thaHomey - #[4492]

SEASON 2 - Common:
110 - 404Billy - #[1060, 1269, 1109]
82 - So La Lune - #[1280, 884]
118 - Baby Neelou - #[1061]
108 - Selug & $enar - #[924]
100 - BabySolo33 - #[1280, 921, 886, 1143]
86 - Winnterzuko - #[1050, 1182]

SEASON 2 - Holo:



<h3> REAL RULEDEX </h3>